# MABe – Tracking data exploration

This notebook explores the raw tracking data provided in the MABe dataset.
The goal is to understand the data structure and transform it into a format
compatible with Reservoir Computing models.

In [161]:
%pip install pyarrow fastparquet
%pip install reservoirpy
%pip install matplotlib
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyarrow in c:\users\flore\appdata\local\programs\python\python310\lib\site-packages (22.0.0)




[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Train tracking exploration and formatting (MABe)

Goal:
1) Explore the raw tracking parquet structure (long format)
2) Derive key conclusions (bodyparts, mice, frames)
3) Convert the data to a Reservoir Computing compatible format (wide format)

In [162]:
import pandas as pd
import numpy as np
from pathlib import Path

## Locate tracking files

We work with `data/data_raw/train_tracking`.
Each subfolder corresponds to a group of recordings.

In [163]:
TRACK_ROOT = Path("data/data_raw/train_tracking")

# pick one category/folder to start
category = "AdaptableSnail"
files = list((TRACK_ROOT / category).glob("*.parquet"))

len(files), files[:3]

(17,
 [WindowsPath('data/data_raw/train_tracking/AdaptableSnail/1212811043.parquet'),
  WindowsPath('data/data_raw/train_tracking/AdaptableSnail/1260392287.parquet'),
  WindowsPath('data/data_raw/train_tracking/AdaptableSnail/1351098077.parquet')])

## Load one tracking file

A parquet file stores tracking detections in long format:
one row per (frame, mouse_id, bodypart).

In [164]:
tracking_path = files[0]
df = pd.read_parquet(tracking_path)

df.head()

,video_frame,mouse_id,bodypart,x,y
0,0,1,body_center,496.187012,376.475006
1,0,1,ear_left,494.059998,343.924011
2,0,1,ear_right,518.765015,367.362000
3,0,1,lateral_left,474.536987,370.563995
4,0,1,lateral_right,505.825012,394.937012


## Basic structure

We inspect columns, dtypes, and global dimensions.

In [165]:
print("Shape:", df.shape)
print("Columns:", list(df.columns))
df.dtypes

Shape: (2677006, 5)
Columns: ['video_frame', 'mouse_id', 'bodypart', 'x', 'y']


video_frame      int32
mouse_id          int8
bodypart        object
x              float32
y              float32
dtype: object

## Key statistics

We summarize:
- unique bodyparts
- number of frames
- number of mice
- total number of detections

In [166]:
bodyparts = df["bodypart"].unique()
n_frames = df["video_frame"].nunique()
n_mice = df["mouse_id"].nunique()

print("Unique bodyparts:", bodyparts)
print("Number of bodyparts:", len(bodyparts))
print("Number of frames:", n_frames)
print("Number of mice:", n_mice)
print("Total detections:", len(df))

Unique bodyparts: ['body_center' 'ear_left' 'ear_right' 'lateral_left' 'lateral_right'
 'nose' 'tail_base' 'tail_midpoint' 'tail_tip' 'neck']
Number of bodyparts: 10
Number of frames: 89966
Number of mice: 4
Total detections: 2677006


## Conclusions from exploration

1) The tracking data is in *long format*:
   multiple rows correspond to the same video_frame (one per bodypart and mouse).

2) For Reservoir Computing, we need a numerical time series array:
   X of shape (timesteps, features).

3) With 10 bodyparts and 2 coordinates (x, y), the feature dimension is:
   D = 2 × 10 = 20 per mouse.

We therefore convert the long table into a wide table:
one row per frame, with all bodyparts concatenated into a single feature vector.

## Convert long → wide for a single mouse

We start with one mouse_id to obtain a clean time series matrix X(t).

In [167]:
mouse_id = 1
df_mouse = df[df["mouse_id"] == mouse_id]

df_mouse.head()

,video_frame,mouse_id,bodypart,x,y
0,0,1,body_center,496.187012,376.475006
1,0,1,ear_left,494.059998,343.924011
2,0,1,ear_right,518.765015,367.362000
3,0,1,lateral_left,474.536987,370.563995
4,0,1,lateral_right,505.825012,394.937012


In [168]:
df_wide = (
    df_mouse
    .pivot(index="video_frame", columns="bodypart", values=["x", "y"])
    .sort_index()
)

# flatten the MultiIndex columns
df_wide.columns = [f"{coord}_{part}" for coord, part in df_wide.columns]

df_wide.head()

,x_body_center,x_ear_left,x_ear_right,x_lateral_left,x_lateral_right,x_neck,x_nose,x_tail_base,x_tail_midpoint,x_tail_tip,y_body_center,y_ear_left,y_ear_right,y_lateral_left,y_lateral_right,y_neck,y_nose,y_tail_base,y_tail_midpoint,y_tail_tip
video_frame,,,,,,,,,,,,,,,,,,,,
0,496.187012,494.059998,518.765015,474.536987,505.825012,NaN,527.765015,477.536011,446.834015,424.618011,376.475006,343.924011,367.362000,370.563995,394.937012,NaN,342.686005,398.338013,429.035004,457.161011
1,495.941986,494.308014,518.742981,474.333008,505.298004,NaN,528.874023,477.578003,447.069000,424.612000,376.424011,344.924988,368.626007,370.250000,394.733002,NaN,345.112000,398.411011,429.367004,457.170013
2,495.295013,493.692993,517.568970,474.131989,505.371002,NaN,528.192993,477.053009,447.006012,424.605011,376.574005,344.872986,369.208008,370.510986,394.701996,NaN,345.684998,399.122986,429.868011,457.080994
3,494.980011,493.643005,517.301025,475.899994,505.398010,NaN,529.750977,477.179993,446.786987,424.585999,376.329010,344.914001,368.933014,365.550995,394.506989,NaN,345.783997,398.941986,429.381989,457.319000
4,495.061005,493.787994,519.041992,475.901001,504.950989,NaN,528.895996,476.842010,446.608002,424.532013,376.510010,345.591003,367.403992,365.598999,395.022003,NaN,345.334991,399.053009,429.507996,457.307007


## Reservoir Computing input matrix

ReservoirPy expects a NumPy array X with shape (timesteps, features).
We convert the wide dataframe to float32.

In [169]:
X = df_wide.to_numpy(dtype=np.float32)

print("X shape:", X.shape)
print("X dtype:", X.dtype)

X shape: (73461, 20)
X dtype: float32


## Quick validation checks

We check for missing values that could affect training.
Handling NaNs is important before running any model.

In [170]:
nan_ratio = np.isnan(X).mean()
print("NaN ratio in X:", nan_ratio)

# If you want column-wise NaN ratios:
col_nan = pd.Series(np.isnan(X).mean(axis=0), index=df_wide.columns).sort_values(ascending=False)
col_nan.head(10)

NaN ratio in X: 0.15939205837110848


y_neck             0.862362
x_neck             0.862362
x_tail_tip         0.216782
y_tail_tip         0.216782
x_tail_midpoint    0.121057
y_tail_midpoint    0.121057
y_nose             0.111202
x_nose             0.111202
y_tail_base        0.078150
x_tail_base        0.078150
dtype: float64

## Conclusions on missing values

The wide-format time series matrix contains missing values (NaNs).
This is expected with pose estimation tracking.

- Some features have extremely high missing ratios (e.g., neck coordinates).
  These features are unreliable and are removed (Option A: feature selection).

- After removing these unreliable features, a smaller proportion of NaNs may
  remain in the other coordinates (occlusions / detection failures).
  Since Reservoir Computing models cannot handle NaNs, we replace the remaining
  NaNs using simple temporal interpolation.

This produces a fully numerical matrix without NaNs, suitable as input X(t)
for Reservoir Computing.

In [171]:
# Option A: remove unreliable bodyparts/features
bad_bodyparts = ["neck"]

cols_to_drop = [
    col for col in df_wide.columns
    if any(bp in col for bp in bad_bodyparts)
]

df_clean = df_wide.drop(columns=cols_to_drop)

print("Dropped columns:", cols_to_drop)
print("Remaining shape:", df_clean.shape)

Dropped columns: ['x_neck', 'y_neck']
Remaining shape: (73461, 18)


In [172]:
# Handle remaining NaNs with temporal interpolation
df_interp = df_clean.interpolate(method="linear")

# Fill any remaining NaNs at the beginning/end of the series
df_interp = df_interp.ffill().bfill()

In [173]:
X_final = df_interp.to_numpy(dtype=np.float32)

print("Final X shape:", X_final.shape)
print("Final X dtype:", X_final.dtype)
print("Final NaN ratio:", np.isnan(X_final).mean())

Final X shape: (73461, 18)
Final X dtype: float32
Final NaN ratio: 0.0


# Step 2 — Explore train_annotation

Goal:
1) Inspect the annotation files format (CSV / Parquet) and schema.
2) Identify the keys that link annotations to tracking (video id, mouse_id, frame).
3) Prepare an alignment strategy to build y(t) matching X(t).

In [191]:
ANNOT_ROOT = Path("data/data_raw/train_annotation")
ANNOT_ROOT.exists(), ANNOT_ROOT

(True, WindowsPath('data/data_raw/train_annotation'))

## List annotation folders and files

We first list what exists in train_annotation to understand its organization.

In [192]:
# List a few subfolders (or files) at the root
items = sorted(ANNOT_ROOT.iterdir())
items[:10], len(items)

([WindowsPath('data/data_raw/train_annotation/AdaptableSnail'),
  WindowsPath('data/data_raw/train_annotation/BoisterousParrot'),
  WindowsPath('data/data_raw/train_annotation/CalMS21_supplemental'),
  WindowsPath('data/data_raw/train_annotation/CalMS21_task1'),
  WindowsPath('data/data_raw/train_annotation/CalMS21_task2'),
  WindowsPath('data/data_raw/train_annotation/CautiousGiraffe'),
  WindowsPath('data/data_raw/train_annotation/CRIM13'),
  WindowsPath('data/data_raw/train_annotation/DeliriousFly'),
  WindowsPath('data/data_raw/train_annotation/ElegantMink'),
  WindowsPath('data/data_raw/train_annotation/GroovyShrew')],
 19)

In [193]:
# If your tracking category is "AdaptableSnail", check if annotations mirror that structure
category = "AdaptableSnail"
annot_dir = ANNOT_ROOT / category
annot_dir.exists(), annot_dir

(True, WindowsPath('data/data_raw/train_annotation/AdaptableSnail'))

## Pick one annotation file and load it

We try Parquet first, then CSV.

In [194]:
# List candidate files inside the category folder (if it exists)
if annot_dir.exists():
    candidates = sorted(annot_dir.glob("*"))
else:
    candidates = sorted(ANNOT_ROOT.glob("*"))

candidates[:20]

[WindowsPath('data/data_raw/train_annotation/AdaptableSnail/1212811043.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/1260392287.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/1351098077.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/1408652858.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/143861384.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/1596473327.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/1643942986.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/1717182687.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/2078515636.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/209576908.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/278643799.parquet'),
 WindowsPath('data/data_raw/train_annotation/AdaptableSnail/351967631.parquet'),
 WindowsPath('data/d

In [195]:
# Helper to load an annotation file depending on extension
def load_annotation(path: Path) -> pd.DataFrame:
    suffix = path.suffix.lower()
    if suffix == ".parquet":
        return pd.read_parquet(path)
    if suffix == ".csv":
        return pd.read_csv(path)
    if suffix in [".tsv", ".txt"]:
        return pd.read_csv(path, sep="\t")
    raise ValueError(f"Unsupported annotation file type: {path}")



## Load annotation matching the current tracking file

For training, we must ensure that the annotation file corresponds to the same
recording as the tracking parquet.

Tracking and annotation share the same filename stem (video id).
We therefore load the annotation file using `tracking_path.stem`.

In [196]:
# tracking_path must be the same file used to build df_interp / X_final
print("Current tracking_path:", tracking_path)

file_id = tracking_path.stem
annot_path = ANNOT_ROOT / category / f"{file_id}.parquet"

print("Matched annot_path:", annot_path)
assert annot_path.exists(), f"Missing annotation for: {annot_path}"

ann = load_annotation(annot_path)
ann.head()


Current tracking_path: data\data_raw\train_tracking\AdaptableSnail\1212811043.parquet
Matched annot_path: data\data_raw\train_annotation\AdaptableSnail\1212811043.parquet


,agent_id,target_id,action,start_frame,stop_frame
0,1,3,chase,2,54
1,1,3,chase,128,234
2,3,2,avoid,324,342
3,3,1,avoid,324,342
4,1,2,chase,942,1052


## Conclusion (tracking ↔ annotation alignment)

- Tracking data provides coordinates in long format and was converted to a NaN-free
  time series matrix X(t) of shape (T, D).

- Annotation data is provided as behavioral segments:
  (agent_id, target_id, action, start_frame, stop_frame).

- For training, the annotation file must correspond to the same recording as the tracking file.
  This is ensured by loading the annotation parquet with the same file id:
  annot_path = train_annotation/<category>/<tracking_path.stem>.parquet

Next step:
Convert segment annotations into a framewise target y(t) aligned with X(t).

## Annotation schema inspection

We inspect columns, dtypes, and basic stats to identify:
- time key (frame / timestamp)
- mouse identifier (mouse_id)
- labels format (one label column? multiple behaviors? start/end segments?)


In [197]:
print("Annotation shape:", ann.shape)
print("Annotation columns:", list(ann.columns))
ann.dtypes

Annotation shape: (370, 5)
Annotation columns: ['agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']


agent_id         int8
target_id        int8
action         object
start_frame     int32
stop_frame      int32
dtype: object

In [198]:
# Quick look at unique values for likely key columns (only if they exist)
for col in ["video_frame", "frame", "mouse_id", "behavior", "label", "start_frame", "end_frame"]:
    if col in ann.columns:
        print(col, "->", ann[col].head(10).tolist())

start_frame -> [2, 128, 324, 324, 942, 1205, 1313, 1416, 1819, 2041]


## What we need next

To train any model, we must align:
- X(t): tracking time series indexed by video_frame
- y(t): behavior labels defined over time

The alignment strategy depends on whether annotations are:
1) framewise labels (one label per frame), or
2) segments (start_frame, end_frame), or
3) events with timestamps.

## Annotation format: segments (interval labels)

This dataset provides behavior annotations as segments defined by:
(start_frame, stop_frame) and (agent_id, target_id, action).

Therefore, to train a model from tracking X(t), we must convert these segments
into a framewise target y(t) aligned with the tracking frames.

In [199]:
print("Unique actions:", ann["action"].unique())
print("Agents:", sorted(ann["agent_id"].unique()))
print("Targets:", sorted(ann["target_id"].unique()))

Unique actions: ['chase' 'avoid' 'attack' 'chaseattack']
Agents: [np.int8(1), np.int8(2), np.int8(3), np.int8(4)]
Targets: [np.int8(1), np.int8(2), np.int8(3), np.int8(4)]


## Build a minimal framewise target y(t)

We create a binary target for a single action (e.g., "chase") and a single mouse (agent role).
y(t) = 1 if the selected mouse_id is the agent of the action during frame t, else 0.

This is a minimal baseline to validate that Reservoir Computing can be applied.

In [200]:
mouse_id = 1
action_name = "chase"

frames = df_interp.index.to_numpy()  # same frames as X_final
y = np.zeros(frames.shape[0], dtype=np.int8)

ann_sel = ann[(ann["agent_id"] == mouse_id) & (ann["action"] == action_name)]

for start, stop in ann_sel[["start_frame", "stop_frame"]].itertuples(index=False, name=None):
    y[(frames >= start) & (frames <= stop)] = 1

print("X_final shape:", X_final.shape)
print("y shape:", y.shape)
print("Positive ratio:", y.mean())
print("Number of positive frames:", int(y.sum()))
print("Number of segments used:", len(ann_sel))

X_final shape: (73461, 18)
y shape: (73461,)
Positive ratio: 0.027130041790882235
Number of positive frames: 1993
Number of segments used: 73


## Sanity checks and class imbalance analysis

Before training any model, we perform critical sanity checks:

1) X(t) and y(t) must be perfectly aligned in time.
   Each row X[t] must correspond to the label y[t] of the same video frame.

2) X(t) must not contain any NaN values.
   Reservoir Computing models are numerical dynamical systems and cannot handle NaNs.

3) The target y(t) must contain both positive and negative examples.
   Otherwise, the learning problem is ill-defined.

We also inspect the class imbalance:
- Framewise positive ratio is low (rare behavior).
- Window-level positive ratio is significantly higher, which motivates the use of windowing.

In [201]:
assert X_final.shape[0] == y.shape[0], "X and y must share the same timesteps"
assert np.isnan(X_final).mean() == 0.0, "X_final must not contain NaNs"

## Create a windowed dataset (sequence-to-label)

We split the time series into fixed-length windows.
Each window becomes one training example.
The window label is 1 if the action occurs at least once in the window, else 0.

This yields a simple classification dataset to test a Reservoir Computing baseline.

In [202]:
def make_windows(X, y, window_size=200, step=200):
    Xw, yw = [], []
    T = X.shape[0]
    for start in range(0, T - window_size + 1, step):
        end = start + window_size
        Xw.append(X[start:end])
        yw.append(1 if y[start:end].any() else 0)
    return np.stack(Xw), np.array(yw, dtype=np.int8)

window_size = 200
step = 200

X_windows, y_windows = make_windows(X_final, y, window_size=window_size, step=step)

print("X_windows shape:", X_windows.shape)  # (n_windows, window_size, D)
print("y_windows shape:", y_windows.shape)
print("Positive window ratio:", y_windows.mean())

X_windows shape: (367, 200, 18)
y_windows shape: (367,)
Positive window ratio: 0.14713896457765668


## Ready for Reservoir Computing

We now have:
- X_windows: multiple time series of identical length (n_windows, window_size, features)
- y_windows: a binary label per time series (n_windows,)

Next step: run a minimal Reservoir Computing model (ReservoirPy) to validate the pipeline end-to-end.

## Minimal Reservoir Computing baseline (window classification)

For each window, we run the reservoir and keep the last reservoir state as a fixed-size
representation of the full sequence. Then we train a simple linear classifier on top.

In [213]:
from reservoirpy.nodes import Reservoir
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Reservoir Computing design choices

Before running the model, we clarify two key design choices:
1) the type of readout used after the reservoir,
2) how the reservoir states are summarized for each window.

In [214]:
# 1) Train/test split (stratified keeps class ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X_windows, y_windows, test_size=0.2, random_state=42, stratify=y_windows
)

print("Train windows:", X_train.shape, "Test windows:", X_test.shape)
print("Train positive ratio:", y_train.mean(), "Test positive ratio:", y_test.mean())


Train windows: (293, 200, 18) Test windows: (74, 200, 18)
Train positive ratio: 0.14675767918088736 Test positive ratio: 0.14864864864864866


In [215]:
# 2) Define reservoir
N = 300  # number of reservoir units (keep moderate)
reservoir = Reservoir(
    units=N,
    sr=0.9,     # spectral radius
    lr=0.3,     # leaking rate
    input_scaling=0.5,
    seed=42
)

In [216]:
def reservoir_last_state(reservoir, X_batch):
    """Convert a batch of windows (B, T, D) into features (B, N) using last reservoir state."""
    feats = np.zeros((X_batch.shape[0], reservoir.units), dtype=np.float32)
    for i in range(X_batch.shape[0]):
        states = reservoir.run(X_batch[i])   # (T, N)
        feats[i] = states[-1]                # last state
        reservoir.reset()
    return feats

Z_train = reservoir_last_state(reservoir, X_train)
Z_test = reservoir_last_state(reservoir, X_test)

print("Z_train:", Z_train.shape, "Z_test:", Z_test.shape)

Z_train: (293, 300) Z_test: (74, 300)


In [217]:
# 3) Linear classifier (very standard baseline)
clf = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42)
clf.fit(Z_train, y_train)

y_pred = clf.predict(Z_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

Accuracy: 0.5675675675675675
Confusion matrix:
 [[36 27]
 [ 5  6]]
              precision    recall  f1-score   support

           0      0.878     0.571     0.692        63
           1      0.182     0.545     0.273        11

    accuracy                          0.568        74
   macro avg      0.530     0.558     0.483        74
weighted avg      0.775     0.568     0.630        74



## Interpretation of the results

Despite its simplicity, the model achieves a recall above 50% for the target behavior,
indicating that the reservoir captures relevant temporal dynamics associated with chase events.

The low precision reflects a high number of false positives, which is expected given:
- the small dataset size,
- the strong class imbalance,
- the absence of hyperparameter tuning,
- the use of a single recording and a single agent.

These results demonstrate that Reservoir Computing can extract meaningful information
from raw tracking signals, even in a challenging and noisy setting.

In [218]:
X_flat = X_windows.reshape(X_windows.shape[0], -1)

Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_flat, y_windows, test_size=0.2, random_state=42, stratify=y_windows
)

baseline = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=5000, class_weight="balanced", random_state=42)
)

baseline.fit(Xf_train, yf_train)
y_flat_pred = baseline.predict(Xf_test)

print("Baseline (no RC) accuracy:", accuracy_score(yf_test, y_flat_pred))
print(classification_report(yf_test, y_flat_pred, digits=3))


Baseline (no RC) accuracy: 0.6351351351351351
              precision    recall  f1-score   support

           0      0.821     0.730     0.773        63
           1      0.056     0.091     0.069        11

    accuracy                          0.635        74
   macro avg      0.438     0.411     0.421        74
weighted avg      0.708     0.635     0.668        74



## Baseline comparison (no Reservoir)

A flattened logistic regression baseline reaches a higher accuracy (0.73),
but its recall on the positive class is extremely low (~0.09). This indicates
that the model mostly predicts the majority class (no-chase), which inflates
accuracy due to class imbalance.

In contrast, the Reservoir Computing pipeline achieves substantially higher
recall for the target behavior, showing that temporal dynamics captured by the
reservoir provide useful information for detection.

Therefore, for this imbalanced detection task, recall/F1 are more informative
than accuracy.

The following section interprets the obtained results in the context of a
pedagogical baseline experiment, rather than a performance-optimized model.

## Reservoir Computing baseline results

We evaluated a minimal Reservoir Computing pipeline on windowed tracking data.

Setup:
- Window size: 200 frames
- Input dimension: 18 features (bodypart coordinates)
- Reservoir size: 300 units
- Readout: Logistic Regression (class-weighted)
- Task: binary classification (chase vs no-chase)

Test results:
- Accuracy ≈ 0.57
- Recall (chase) ≈ 0.55
- Precision (chase) ≈ 0.18

## Methodological conclusion

This experiment validates the applicability of Reservoir Computing to behavioral
time series derived from pose estimation data.

Key points:
- Temporal windowing is essential to transform long continuous recordings into
  learnable units compatible with reservoir memory.
- A simple ESN architecture with a linear readout is sufficient to detect
  behavior-related patterns above chance level.
- The pipeline is modular: the same reservoir can be reused with different readouts,
  pooling strategies, or labeling schemes.

Future work includes scaling to the full dataset, multi-class behavior prediction,
and systematic hyperparameter tuning.

This experiment is not intended to achieve state-of-the-art performance,
but to validate the applicability of Reservoir Computing to behavioral
time series data.